In [7]:
import torch
from torch.autograd import Variable
from torch.nn import functional as F
import torchvision.transforms as transforms
import torchvision.datasets as dsets

In [14]:
INPUT_DIM = 784
OUTPUT_DIM = 10
LR_RATE = 0.001
BATCH_SIZE = 100
N_ITERS = 3000
EPOCHS = N_ITERS / (len(train_data) / BATCH_SIZE)

In [15]:
train_data = dsets.MNIST(root="./data", train=True,
                         transform = transforms.ToTensor(), download=True)

test_data = dsets.MNIST(root="./data", train=False, transform=transforms.ToTensor())

In [27]:
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=False)

In [28]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        y_pred = self.linear(x)
        return y_pred
    
model = LogisticRegression(INPUT_DIM, OUTPUT_DIM)

In [29]:
criterion = torch.nn.CrossEntropyLoss() # computes softmax and then the cross entropy

optimizer = torch.optim.SGD(model.parameters(), lr=LR_RATE)

In [31]:
iterr = 0
for epoch in range(int(EPOCHS)):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        iterr += 1
        if iterr % 500 == 0:
            # calculating accuracy
            correct = 0
            total = 0
            for images, labels in test_loader:
                images = Variable(images.view(-1, 28*28))
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum()
            accuracy = 100 * correct / total
            print("Iteration: {}, Loss: {}, Accuracy: {}".format(iterr, loss.item(), accuracy))

Iteration: 500, Loss: 0.8605776429176331, Accuracy: 83.6500015258789
Iteration: 1000, Loss: 0.8697571754455566, Accuracy: 83.9800033569336
Iteration: 1500, Loss: 0.762584388256073, Accuracy: 84.37999725341797
Iteration: 2000, Loss: 0.8713825345039368, Accuracy: 84.73999786376953
Iteration: 2500, Loss: 0.7044221758842468, Accuracy: 85.12000274658203
Iteration: 3000, Loss: 0.8354395031929016, Accuracy: 85.45999908447266
